In [0]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import sys
import os

In [0]:
rank = spark.read.parquet('dbfs:/mnt/lsde/group24/subreddit_rank_2')
sub_list = rank.select(col('subreddit')).orderBy(desc('total_num_comments'),desc('total_score')) \
                                              .rdd.map(lambda r: r.subreddit).collect()

In [0]:
sub_list

Out[3]: ['AskReddit',
 'memes',
 'politics',
 'teenagers',
 'wallstreetbets',
 'AmItheAsshole',
 'FreeKarma4U',
 'worldnews',
 'nfl',
 'nba',
 'soccer',
 'news',
 'PublicFreakout',
 'CryptoCurrency',
 'relationship_advice',
 'unpopularopinion',
 'dankmemes',
 'Superstonk',
 'gaming',
 'funny',
 'pics',
 'SquaredCircle',
 'PoliticalCompassMemes',
 'interestingasfuck',
 'conspiracy',
 'leagueoflegends',
 'nextfuckinglevel',
 'RedditSessions',
 'CFB',
 'distantsocializing',
 'neoliberal',
 'Genshin_Impact',
 'NoStupidQuestions',
 'aww',
 'Market76',
 'movies',
 'antiwork',
 'formula1',
 'facepalm',
 'todayilearned',
 'AskMen',
 'pcmasterrace',
 'gonewild',
 'Minecraft',
 'FreeKarma4You',
 'Showerthoughts',
 'apexlegends',
 'hockey',
 'FortNiteBR',
 'WhitePeopleTwitter',
 'amcstock',
 'DestinyTheGame',
 'MMA',
 'HolUp',
 'fantasyfootball',
 'shitposting',
 'KGBTR',
 'mildlyinfuriating',
 'mildlyinteresting',
 'Conservative',
 'Eldenring',
 'Damnthatsinteresting',
 'anime',
 'baseball',
 'C

In [0]:
file_path = 'dbfs:/FileStore/Group24/expertise_json/subreddit='

In [0]:
sub_list[0]

Out[5]: 'AskReddit'

In [0]:
#This function is to union each subreddit json file to a large json file
def union_json(first_json, sub_list):
    result  = first_json
    for sub in sub_list:
        df = spark.read.json(file_path+ sub +'/*.json') \
                    .withColumn('subreddit',lit(sub))
        print(file_path+ sub +'/*.json')
        result = result.union(df)
    return result
first_json = spark.read.json(file_path+sub_list[0]+'/*.json') \
                    .withColumn('subreddit',lit(sub_list[0]))
total_json = union_json(first_json, sub_list[1:])

dbfs:/FileStore/Group24/expertise_json/subreddit=memes/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=politics/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=teenagers/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=wallstreetbets/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=AmItheAsshole/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=FreeKarma4U/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=worldnews/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=nfl/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=nba/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=soccer/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=news/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=PublicFreakout/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=CryptoCurrency/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=relationship_advice/*.json
dbfs:/FileStore/Group24/expertise_json/subreddit=unpopularopinion/*.js

In [0]:
total_json.show()

+--------------------+------------+--------------------+----+------------+---------+
|              author|expert_score|           expertise|rank|subreddit_id|subreddit|
+--------------------+------------+--------------------+----+------------+---------+
|     TannedCroissant|     4100664|[{life stops givi...|   1|    t5_2qh1i|AskReddit|
|             -eDgAR-|     3026721|[{outdoor movie n...|   2|    t5_2qh1i|AskReddit|
|           Back2Bach|     2986169|[{isabella stewar...|   3|    t5_2qh1i|AskReddit|
| Poem_for_your_sprog|     2415792|[{big fucking mes...|   4|    t5_2qh1i|AskReddit|
|      DeathSpiral321|     2044027|[{credit card num...|   5|    t5_2qh1i|AskReddit|
|           poopellar|     1826440|[{bean showing te...|   6|    t5_2qh1i|AskReddit|
|   A_Very_Brave_Taco|     1764937|[{savory pastries...|   7|    t5_2qh1i|AskReddit|
|            elee0228|     1269129|[{fucking magical...|   8|    t5_2qh1i|AskReddit|
|          Portarossa|     1242414|[{real stick-in-t...|   9|    

In [0]:
total_json = total_json.select(col('rank'), col('subreddit'), col('subreddit_id')
                               , col('author'), col('expert_score'), col('expertise'))

In [0]:
total_json.coalesce(1).write.mode('overwrite') \
                .option("mapreduce.fileoutputcommitter.marksuccessfuljobs","false") \
                .json('dbfs:/FileStore/Group24/final_expertise_json')

In [0]:
total_json.show()

+----+---------+------------+--------------------+------------+--------------------+
|rank|subreddit|subreddit_id|              author|expert_score|           expertise|
+----+---------+------------+--------------------+------------+--------------------+
|   1|AskReddit|    t5_2qh1i|     TannedCroissant|     4100664|[{life stops givi...|
|   2|AskReddit|    t5_2qh1i|             -eDgAR-|     3026721|[{outdoor movie n...|
|   3|AskReddit|    t5_2qh1i|           Back2Bach|     2986169|[{isabella stewar...|
|   4|AskReddit|    t5_2qh1i| Poem_for_your_sprog|     2415792|[{big fucking mes...|
|   5|AskReddit|    t5_2qh1i|      DeathSpiral321|     2044027|[{credit card num...|
|   6|AskReddit|    t5_2qh1i|           poopellar|     1826440|[{bean showing te...|
|   7|AskReddit|    t5_2qh1i|   A_Very_Brave_Taco|     1764937|[{savory pastries...|
|   8|AskReddit|    t5_2qh1i|            elee0228|     1269129|[{fucking magical...|
|   9|AskReddit|    t5_2qh1i|          Portarossa|     1242414|[{